ours vs monte carlo vs roth's (vary sigma and num_layers)

In [ ]:
import torch
import torch.nn as nn
from experiments import GaussianExp


def relative_error(x, y):
    return 2 * (x - y).abs() / (x.abs() + y.abs())


def get_model(layers, batch_norm=True):
    if isinstance(layers, str):
        exp = lambda: None
        exp.device = 'cpu'
        exp.config = {'model': layers}
        return GaussianExp.__base__.init_model(exp)
    features = []
    for i in range(layers):
        features.append(nn.Conv2d(1 if i == 0 else 16, 16, 3))
        if batch_norm:
            features.append(nn.BatchNorm2d(16))
        features.append(nn.ReLU(inplace=True))
    classifier = [nn.AdaptiveAvgPool2d(1), nn.Flatten(), nn.Linear(16, 10)]
    return nn.Sequential(*features, *classifier)


def channel_sigma(sigma):
    return torch.tensor((sigma,)) #/ torch.tensor((0.2470, 0.2435, 0.2616)).view(-1, 1, 1)


def gaussian_forward(model, images, sigma):
    exp = lambda: None
    exp.model = model
    exp.sigma = channel_sigma(sigma).to(images.device, images.dtype)
    exp.relu_mean = GaussianExp.relu_mean
    exp.conv2d_var_mean = GaussianExp.conv2d_var_mean
    exp.batch_norm_var_mean = GaussianExp.batch_norm_var_mean
    mu_forward, forward = GaussianExp.gaussian_forward(exp, {'inputs': images}, False)
    return mu_forward, forward


@torch.no_grad()
def full_gaussian_forward(model, image, sigma, samples=1000):
    meanf, f = gaussian_forward(model, image.unsqueeze(0), sigma)
    noisy = image.new_empty(samples, *image.size()).normal_()
    noisy.mul_(channel_sigma(sigma).to(image.device, image.dtype)).add_(image)
    std, mean = torch.std_mean(model(noisy), dim=0)
    return dict(std_monte_carlo=std, mu_monte_carlo=mean, mu_forward=meanf.squeeze(0), forward=f.squeeze(0))

In [ ]:
from collections import defaultdict


results = defaultdict(dict)
image = torch.randn(1, 10, 10)
for layers in range(1, 4 + 1):
    model = get_model(layers).eval()
    for sigma in [0.01, 0.1, 0.3, 0.5]:
        results[layers][sigma] = full_gaussian_forward(model, image, sigma)

In [ ]:
import matplotlib.pyplot as plt


plt.figure(figsize=(10, 10))
for sigma in [0.01, 0.1, 0.3, 0.5]:
    mean_relative_error = [
        relative_error(results[i][sigma]['mu_forward'], results[i][sigma]['mu_monte_carlo']).mean().item()
        for i in range(1, 4 + 1)
    ]
    plt.plot(range(1, 4 + 1), mean_relative_error, label=f'$\\sigma = {sigma}$')
plt.legend()
plt.yscale('log')
plt.ylabel('average relative error')
plt.xlabel('number of layers')
plt.title('sanity check')
plt.show()

batchnorm?